In [14]:
import implementations
import pandas as pd
import numpy as np
import helpers



## TODO DELETE , pandas isn't allowed , just here for exploratory data annalysis 

x = pd.read_csv("./dataset/x_train.csv",sep=',')
x.head()
x.describe()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
count,328135.000000,328135.000000,3.281350e+05,328135.000000,328135.000000,328135.000000,328135.000000,3.281350e+05,3.281350e+05,188720.0,...,328135.000000,328135.000000,326252.000000,326252.000000,326252.000000,328135.000000,328135.000000,116757.00000,116757.000000,296055.000000
mean,29.973651,6.360690,6.563638e+06,6.416661,14.496235,2015.024758,1115.014857,2.015005e+09,2.015005e+09,1.0,...,3.488567,2.737611,2.717093,2.814279,3.652698,1.820421,1.882390,2.28991,2.406794,1.966675
std,16.031777,3.487263,3.489248e+06,3.492739,8.334495,0.155387,35.721753,4.118475e+03,4.118475e+03,0.0,...,2.395363,2.445086,1.317887,1.348067,1.259353,2.355332,2.346068,2.51678,2.771851,1.434133
min,1.000000,1.000000,1.012016e+06,1.000000,1.000000,2015.000000,1100.000000,2.015000e+09,2.015000e+09,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
25%,19.000000,3.000000,3.232015e+06,3.000000,8.000000,2015.000000,1100.000000,2.015002e+09,2.015002e+09,1.0,...,2.000000,2.000000,2.000000,2.000000,3.000000,1.000000,1.000000,1.00000,1.000000,1.000000
50%,29.000000,6.000000,6.242015e+06,6.000000,14.000000,2015.000000,1100.000000,2.015004e+09,2.015004e+09,1.0,...,3.000000,2.000000,3.000000,3.000000,4.000000,1.000000,1.000000,1.00000,1.000000,2.000000
75%,44.000000,9.000000,1.002202e+07,10.000000,21.000000,2015.000000,1100.000000,2.015007e+09,2.015007e+09,1.0,...,4.000000,2.000000,3.000000,3.000000,4.000000,1.000000,1.000000,2.00000,2.000000,2.000000
max,72.000000,12.000000,1.231202e+07,12.000000,31.000000,2016.000000,1200.000000,2.015023e+09,2.015023e+09,1.0,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000


In [15]:
yb, input_data, ids = helpers.load_csv_data(data_path="./dataset/")

IsADirectoryError: [Errno 21] Is a directory: './dataset/'